In [1]:
!pip install eli5

     |████████████████████████████████| 106 kB 25.3 MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
ls


drive/  sample_data/


In [3]:
cd "/content/drive/MyDrive/Colab Notebooks/dw_transform"

/content/drive/MyDrive/Colab Notebooks/dw_transform


In [10]:
ls data

men_shoes.csv  shoes_prices.csv


In [4]:
df = pd.read_csv("data/men_shoes.csv", low_memory=False)

In [5]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:5

In [6]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values 
  y = df["prices_amountmin"].values 

  scores = cross_val_score(model, X, y, scoring="neg_mean_absolute_error")
  return np.mean(scores), np.std(scores)

In [7]:
df["brand_cat"] = df["brand"].map(lambda x: str(x).lower()).factorize()[0]

In [8]:
run_model(["brand_cat"])

(-58.133398968282776, 4.206122611474276)

In [9]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)

run_model(["brand_cat"], model)

(-57.31783843165656, 4.181246596160967)

In [10]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [11]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [13]:
def parse_features(x):
  if str(x).lower() == "nan": return []
  return literal_eval(x.replace('\\"', '"')) 

df["features_parsed"] = df["features"].map(parse_features)

In [14]:
df["features_parsed"].head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [12]:
def parse_features(x):
  output_dict = {}
  if str(x).lower() == "nan": return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item["key"].lower().strip()
    value = item["value"][0].lower().strip()
    output_dict[key] = value
  
  return output_dict


df["features_parsed"] = df["features"].map(parse_features)

In [13]:
df["features_parsed"].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [14]:
keys = set()

df["features_parsed"].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [15]:
def get_name_feat(key):
  return "feat_" + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/476 [00:00<?, ?it/s]

In [16]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_product id', 'feat_used with equipment type', 'feat_protection',
       'feat_frame shape', 'feat_polarized lenses', 'feat_weight', 'feat_sku#',
       'feat_size', 'feat_nickel free', 'feat_country of origin'],
      dtype='object', length=526)

In [17]:
df[ df["feat_athlete"].isnull() ].shape[0]

18272

In [18]:
df.shape[0]

18280

In [19]:
df[ False == df["feat_athlete"].isnull() ].shape[0] / df.shape[0] *100

0.0437636761487965

In [20]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] *100

In [21]:
keys_stat

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [22]:
{k: v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [39]:
df["feat_brand_cat"] = df["feat_brand"].factorize()[0]
df["feat_color_cat"] = df["feat_color"].factorize()[0]
df["feat_gender_cat"] = df["feat_gender"].factorize()[0]
df["feat_manufacturer_part_number_cat"] = df["feat_manufacturer_part_number"].factorize()[0]
df["feat_material_cat"] = df["feat_material"].factorize()[0]


In [24]:
df["brand"] = df["brand"].map( lambda x: str(x).lower() )
df[ df["brand"] == df.feat_brand ].shape[0]

8846

In [25]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(["brand_cat"], model)

(-57.30336499006842, 4.226622505567565)

In [26]:
feat = ["brand_cat", "feat_brand_cat", "feat_color_cat", "feat_gender_cat", "feat_manufacturer_part_number_cat", "feat_material_cat"]
model = RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(feat, model)

(-57.17972696488427, 4.216065484677586)

In [29]:
X = df[feat].values
y = df["prices_amountmin"].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names = feat)

Weight,Feature
0.2568 ± 0.0106,brand_cat
0.1062 ± 0.0094,feat_material_cat
0.0426 ± 0.0044,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0023 ± 0.0004,feat_manufacturer_part_number_cat


In [30]:
feat = ["brand_cat", "feat_brand_cat", "feat_color_cat", "feat_gender_cat", "feat_material_cat"]
model = RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(feat, model)

(-57.06222110227685, 4.203190525787177)

In [31]:
X = df[feat].values
y = df["prices_amountmin"].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names = feat)

Weight,Feature
0.2581 ± 0.0103,brand_cat
0.1015 ± 0.0115,feat_material_cat
0.0438 ± 0.0020,feat_gender_cat
0.0204 ± 0.0003,feat_brand_cat
0.0065 ± 0.0013,feat_color_cat


In [33]:
feat = ["brand_cat", "feat_brand_cat", "feat_gender_cat", "feat_manufacturer_part_number_cat", "feat_material_cat"]
model = RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(feat, model)

(-57.15195892808133, 4.187044579141901)

In [36]:
df["brand"].value_counts(normalize=True)

nike             0.097210
puma             0.033315
ralph lauren     0.028775
vans             0.021116
new balance      0.020295
                   ...   
dadawen          0.000055
pleaser shoes    0.000055
avia peter       0.000055
deroyal          0.000055
cabela's         0.000055
Name: brand, Length: 1732, dtype: float64

In [37]:
df[ df["brand"] == "nike"].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [40]:

df["feat_sport_cat"] = df["feat_sport"].factorize()[0]
df["feat_style_cat"] = df["feat_style"].factorize()[0]

In [103]:
feat = ["brand_cat", "feat_style_cat", "feat_brand_cat", "feat_color_cat", "feat_gender_cat", "feat_material_cat"]
model = RandomForestRegressor(max_depth=5, n_estimators=100)

result = run_model(feat, model)

In [104]:
X = df[feat].values
y = df["prices_amountmin"].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names = feat)

(-57.01112030683798, 4.2716432626163705)


Weight,Feature
0.2545 ± 0.0108,brand_cat
0.1025 ± 0.0094,feat_material_cat
0.0437 ± 0.0045,feat_gender_cat
0.0201 ± 0.0020,feat_brand_cat
0.0073 ± 0.0004,feat_style_cat
0.0064 ± 0.0010,feat_color_cat


In [110]:
feat = ["brand_cat", "feat_style_cat", "feat_brand_cat", "feat_gender_cat", "feat_material_cat"]
model = RandomForestRegressor(max_depth=5, n_estimators=100)

result = run_model(feat, model)

In [111]:
X = df[feat].values
y = df["prices_amountmin"].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names = feat)

(-57.064200744585364, 4.293313697094456)


Weight,Feature
0.2611 ± 0.0100,brand_cat
0.1038 ± 0.0115,feat_material_cat
0.0449 ± 0.0022,feat_gender_cat
0.0211 ± 0.0008,feat_brand_cat
0.0067 ± 0.0007,feat_style_cat


In [58]:
df[ df["brand"] == "nike"].features_parsed.sample(5).values

array([{'manufacturer part number': '640737-905', 'gender': 'men', 'shoe category': 'athletic & sneakers', 'brand': 'nike', 'color': 'black'},
       {},
       {'gender': 'unisex', 'color': 'cool grey/hyper pink/pure platinum', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '685840060', 'condition': 'new', 'brand': 'nike', 'shoe width': 'medium'},
       {'style': 'athletic', 'country/region of manufacture': 'thailand', 'condition': 'new with tags', 'country//region of manufacture': 'thailand', 'color': 'yellow'},
       {'material': 'synthetic', 'country/region of manufacture': 'vietnam', 'color': 'black'}],
      dtype=object)

In [60]:
for key in keys:
  df[get_name_feat(key) + "_cat"] = df[get_name_feat(key)].factorize()[0]
  

In [61]:
feats_cat = [x for x in df.columns if "cat" in x]
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_shoe category',
 'feat_catalog',
 'feat_fabrication',
 'feat_certifications and listings',
 'feat_clothing category',
 'feat_location - city/state',
 'feat_location - country',
 'feat_multi pack indicator',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer_part_number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_cm_cat',
 'feat_item package quantity_cat',
 'feat_pant style_cat',
 'feat_country of origin - components_cat',
 'feat_contains cubic zirconia_cat',
 'feat_use_cat',
 'feat_shipping weight_cat',
 'feat_reinforced knee_cat',
 'feat_polarized_cat',
 'feat_band width_cat',
 'feat_lens color family_cat',
 'feat_lens material_cat',
 'feat_impact resistant_cat',
 'feat_weather-resistant_cat',
 'feat_vendor description_cat',
 'feat_watch case shape_cat',
 'feat_high visibility (ansi compliant)_cat',
 'feat_style number_cat',
 'feat_frame style_cat',
 'feat_power re

In [193]:
feat = ["brand_cat", "feat_brand_cat", "feat_gender_cat", "feat_material_cat"]
feat += ['feat_manufacturer number_cat',
 'feat_safety features_cat',
 'feat_item weight_cat',
 'feat_shipping to usa_cat',
 'feat_retail price_cat',
 'feat_closure_cat',
 'feat_item character_cat',
 'feat_clasp type_cat',
 'feat_rx-able_cat',
 'feat_ring size_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_country/region of manufacture_cat',
 'feat_crown_cat',
 'feat_charge time (minutes)_cat',
 'feat_clothing category_cat',
 'feat_color mapping_cat',
 'feat_chest_cat',
 'feat_watch style_cat',
 'feat_waist size_cat',
 'feat_sizearm_cat',
 'feat_necklace type_cat',
 'feat_number of pockets_cat',
 'feat_to fit_cat',
 'feat_casing_cat',
 'feat_pocket_cat',
 'feat_character_cat',
 'feat_metal type_cat',
 'feat_handbag style_cat',
 'feat_frame material_cat',
 'feat_lining_cat',
 'feat_location - city/state_cat',
 'feat_water resistant_cat',
 'feat_fit:_cat',
 'feat_sports league_cat',
 'feat_polarised_cat',
 'feat_fit to_cat',
 'feat_fabric_cat',
 'feat_mpn_cat',
 'feat_transactionid_cat',
 'feat_is portable_cat',
 'feat_free item_cat',
 'feat_bed size_cat',
 'feat_theme_cat',
 'feat_location - country_cat',
 'feat_product type_cat',
 'feat_jewelry setting_cat',
 'feat_fastener type_cat',
 'feat_purpose_cat',
 'feat_compatible devices_cat',
 'feat_has mercury_cat',
 'feat_country of manufacturer_cat',
 'feat_domestic shipping_cat',
 'feat_international shipping?_cat',
 'feat_audience_cat',
 'feat_lens socket width_cat',
 'feat_military style_cat',
 'feat_international shipping_cat',
 'feat_sock style_cat',
 'feat_item type_cat',
 'feat_labor_cat',
 'feat_sub style_cat',
 'feat_batteries required_cat',
 'feat_stability_cat',
 'feat_is recyclable_cat',
 'feat_pronation_cat',
 'feat_country////region of manufacture_cat',
 'feat_stylenumber_cat',
 'feat_bezel_cat',
 'feat_number of compartments_cat',
 'feat_autographed_cat',
 'feat_fishing_cat',
 'feat_hooded_cat',
 'feat_szie_cat',
 'feat_chain/necklace length (in.)_cat',
 'feat_gemstone type_cat',
 'feat_attachment_cat',
 'feat_smart watch_cat',
 'feat_sizebridge_cat',
 'feat_is waterproof_cat',
 'feat_colour code_cat',
 'feat_ground_cat',
 'feat_eye protection type_cat',
 'feat_safety apparel product type_cat',
 'feat_width_cat',
 'feat_stone type_cat',
 'feat_is water-resistant_cat',
 'feat_licensed_cat',
 'feat_stock no._cat',
 'feat_type 2_cat',
 'feat_gold karat_cat',
 'feat_includes_cat',
 'feat_is wheeled_cat',
 'feat_digital camera_cat',
 'feat_materials_cat',
 'feat_fits styles_cat',
 'feat_case type:_cat',
 'feat_shoe size_cat',
 'feat_boot height_cat',
 'feat_athlete_cat',
 'feat_compass_cat',
 'feat_has paper wood_cat',
 'feat_product #_cat',
 'feat_multi pack indicator_cat',
 'feat_pattern or design_cat',
 'feat_lens tint_cat',
 'feat_age range_cat',
 'feat_product id_cat',
 'feat_used with equipment type_cat',
 'feat_protection_cat',
 'feat_frame shape_cat',
 'feat_polarized lenses_cat',
 'feat_weight_cat',
 'feat_sku#_cat',
 'feat_size_cat',
 'feat_nickel free_cat',
 'feat_country of origin_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)

result = run_model(feat, model)

In [194]:
X = df[feat].values
y = df["prices_amountmin"].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names = feat)

(-57.192844631524295, 4.189299211158535)


Weight,Feature
0.2506 ± 0.0091,brand_cat
0.0985 ± 0.0121,feat_material_cat
0.0360 ± 0.0021,feat_gender_cat
0.0169 ± 0.0021,feat_brand_cat
0.0129 ± 0.0012,feat_weight_cat
0.0029 ± 0.0006,feat_frame material_cat
0.0021 ± 0.0006,feat_metal type_cat
0.0020 ± 0.0008,feat_gold karat_cat
0.0013 ± 0.0002,feat_fastener type_cat
0.0011 ± 0.0004,feat_country/region of manufacture_cat


In [112]:
feat = ["brand_cat", "feat_style_cat", "feat_brand_cat", "feat_color_cat", "feat_gender_cat", "feat_material_cat"]
model = RandomForestRegressor(max_depth=5, n_estimators=100)

result = run_model(feat, model)

In [113]:
X = df[feat].values
y = df["prices_amountmin"].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names = feat)

(-56.99413729170302, 4.253539181608901)


Weight,Feature
0.2545 ± 0.0108,brand_cat
0.1025 ± 0.0094,feat_material_cat
0.0437 ± 0.0045,feat_gender_cat
0.0201 ± 0.0020,feat_brand_cat
0.0073 ± 0.0004,feat_style_cat
0.0064 ± 0.0010,feat_color_cat


In [203]:
feat = ["brand_cat", "feat_adjustable_cat", "feat_brand_cat", "feat_material_cat", "feat_case thickness_cat", "feat_movement_cat", "feat_fabric content_cat", "feat_weight_cat"]
model = RandomForestRegressor(max_depth=5, n_estimators=100)

result = run_model(feat, model)

In [204]:
X = df[feat].values
y = df["prices_amountmin"].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names = feat)

(-57.07890471453965, 4.202600956447894)


Weight,Feature
0.2569 ± 0.0074,brand_cat
0.1166 ± 0.0143,feat_material_cat
0.0385 ± 0.0122,feat_adjustable_cat
0.0254 ± 0.0030,feat_brand_cat
0.0182 ± 0.0010,feat_fabric content_cat
0.0131 ± 0.0014,feat_weight_cat
0.0094 ± 0.0014,feat_case thickness_cat
0.0056 ± 0.0007,feat_movement_cat


In [206]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   transform_one/day_4.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [207]:
!git add .

In [210]:
!git commit -m "day_4.ipynb modified"

[main 4658d43] day_4.ipynb modified
 1 file changed, 1 insertion(+), 2 deletions(-)
 rewrite transform_one/day_4.ipynb (94%)


In [209]:
!git config --global user.email "paulina.w.rakowska@gmail.com"
!git config --global user.name "Paulina Rakowska"